# Librery

In [85]:
# Import the libraries
import numpy as np
import matplotlib.pyplot as plt  # for 畫圖用
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense,  Conv1D, MaxPooling1D, Flatten
from tensorflow.keras import optimizers

# Make Dictionary for One-hot Encoding

In [2]:
# Import the training set
train_log = pd.read_csv('/user_data/my/作業/data/train_logs.csv')  # 讀取訓練集
train_log = train_log[["id","down_event"]]
train_log

,id,down_event
0,001519c8,Leftclick
1,001519c8,Leftclick
2,001519c8,Shift
3,001519c8,q
4,001519c8,q
...,...,...
8405893,fff05981,Leftclick
8405894,fff05981,Shift
8405895,fff05981,q
8405896,fff05981,Leftclick


In [3]:
train_log["down_event"][0]

'Leftclick'

In [4]:
train_score = pd.read_csv('/user_data/my/作業/data/train_scores.csv')  # 讀取訓練集label
train_score = train_score.set_index("id")
train_score

,score
id,
001519c8,3.5
0022f953,3.5
0042269b,6.0
0059420b,2.0
0075873a,4.0
...,...
ffb8c745,3.5
ffbef7e5,4.0
ffccd6fd,1.5


In [5]:
# Import the training set
test_log = pd.read_csv('/user_data/my/作業/data/test_logs.csv')  # 讀取測試集
test_log

,id,event_id,down_time,up_time,action_time,activity,down_event,up_event,text_change,cursor_position,word_count
0,0000aaaa,1,338433,338518,85,Input,Space,Space,,0,0
1,0000aaaa,2,760073,760160,87,Input,Space,Space,,1,0
2,2222bbbb,1,711956,712023,67,Input,q,q,q,0,1
3,2222bbbb,2,290502,290548,46,Input,q,q,q,1,1
4,4444cccc,1,635547,635641,94,Input,Space,Space,,0,0
5,4444cccc,2,184996,185052,56,Input,q,q,q,1,1


In [10]:
# see all key
D = train_log['down_event'].unique()
D

array(['Leftclick', 'Shift', 'q', 'Space', 'Backspace', '.', ',', 'Enter',
       'ArrowLeft', "'", ';', 'ArrowRight', '-', '?', 'Tab', '"',
       'ArrowUp', 'ArrowDown', 'Rightclick', '=', 'CapsLock', 'Control',
       'c', 'v', '/', 'Delete', ':', 'z', '[', '$', '(', ')', '+', 'Home',
       'End', '\\', 'Meta', '*', '&', 'AudioVolumeMute', 'x', '!',
       'Insert', 'MediaPlayPause', 'NumLock', '%', 'V', '>', 'Alt',
       'AudioVolumeUp', 'ContextMenu', 'AudioVolumeDown', 'a', '<',
       'PageDown', ']', 'Middleclick', '@', 'F12', 'j', '\x96', 'Dead',
       't', 's', 'n', 'y', '{', 'ScrollLock', '¿', 'Process', '}',
       'MediaTrackPrevious', 'MediaTrackNext', 'F3', '^', 'Unidentified',
       'Cancel', '2', 'i', 'd', 'r', 'e', '`', '\x9b', 'm', '#', '~',
       'PageUp', 'T', 'A', 'b', 'S', 'ModeChange', '_', 'Escape', 'F11',
       'Unknownclick', 'AltGraph', 'F10', 'h', 'F15', 'Clear', 'OS', 'F',
       'C', 'o', 'Ä±', 'f', 'u', 'w', 'p', 'g', 'M', 'l', '|',
       'â\x80\x

In [8]:
np.save("key.npy",D)

In [9]:
# one-hot encoding dictionary
D_cat = to_categorical(range(len(D)))
D_ind = { D[i]:D_cat[i] for i in range(len(D)) }
D_ind[0] = np.zeros(len(D))
D_ind

{'Leftclick': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'Shift': array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

# One-hot Encoding Data

In [8]:
id_class = train_log.groupby('id')
event_per_essay = [ [id, len(id_class.groups[id])] for id in train_log['id'].unique() ]
event_per_essay = np.array(event_per_essay)
event_per_essay

array([['001519c8', '2557'],
       ['0022f953', '2454'],
       ['0042269b', '4136'],
       ...,
       ['ffccd6fd', '3063'],
       ['ffec5b38', '3242'],
       ['fff05981', '3619']], dtype='<U21')

In [11]:
max(event_per_essay[:,1].astype(int))

12876

In [12]:
id_class.groups["001519c8"]

Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
       ...
       2547, 2548, 2549, 2550, 2551, 2552, 2553, 2554, 2555, 2556],
      dtype='int64', length=2557)

In [13]:
np.array(id_class.groups["001519c8"])

array([   0,    1,    2, ..., 2554, 2555, 2556])

In [9]:
train_log.loc[id_class.groups["001519c8"]].loc[0,"down_event"]

'Leftclick'

In [12]:
len(train_log.loc[id_class.groups["001519c8"]])

8405898

In [9]:
def oneHot_eachEvent( df:pd.DataFrame, num_desire:int )->list:
    # 確保 r_df 會有 num_desire 筆 "down_event"
    r_df = df[["down_event"]]
    if(len(df)<num_desire):
        print("enter padding")
        padding = { "down_event":["0"]*(num_desire-len(df)) }
        padding = pd.DataFrame(padding)
        r_df = pd.concat([r_df, padding], axis=0)

    r_df = r_df.reset_index().drop("index", axis=1)

    # 製作 r_log (return value)
    r_log = []
    for i in range(num_desire):
        # print(f"enter event {i}")
        event = r_df.at[i, "down_event"]
        r_log.append(D_ind[event])
        
    return r_log

In [ ]:
train2D = np.zeros((2471, 12876, 132))  #( num_essay, max_event, one-hot encoding)
for ind, [ind_essay, num_event] in enumerate(event_per_essay):
  print(ind_essay,num_event)
  cur_df = train_log.loc[id_class.groups[ind_essay]].reset_index().drop("index", axis=1)
  for ind_event in range(int(num_event)):
    # train2D[ind, ind_event] = DorU_index[
    #       cur_df.loc[ind_event,"down_event"]
    # ]
    try:
      train2D[ind, ind_event] = DorU_index[
          cur_df.loc[ind_event,"down_event"]
      ]
    except:
      print(ind_event)
      display(train_log.loc[id_class.groups[ind_essay]])
train2D

001519c8 2557
0022f953 2454
0042269b 4136
0059420b 1556
0075873a 2531
0081af50 2211
0093f095 1765
009e23ab 2353
00e048f1 1585
00e1f05a 7826
00e713bd 4565
00f0737e 2416
00f8e84c 3721
00fc9a6a 2701
0144e4d5 2117
014e7ae9 3919
015aa732 3171
0178a105 3446
0182aa1c 3126
0190ff4c 1922
01963e20 5016
019737b6 5526
01992d32 3756
01c359fc 2934
01d0ba4b 1639
01d602a7 3573
0249a095 5394
0262bf61 4442
026be946 4338
0294b4f5 1887
02a41d1a 2628
02cf6a52 2050
02d3c9fc 3626
02e86b6a 5437
031c0c58 2713
034d61db 3565
035f09fc 3337
036cfd42 7186
0395b217 3915
03971ddf 5941
040c429b 5763
0417d421 2317
0432f117 3222
0445b534 1697
044b274d 5974
044c5c54 1310
048fd254 2229
04b88b3e 3616
04c5f885 1464
04e37b6b 2067
04e63aec 5906
04f83ccb 5807
052a7811 2587
052b25e1 5290
052cf3a6 1610
053be4e5 4777
05624ab0 1738
056c41fc 1740
05b35fa0 2240
05f425a4 2828
05f8ca37 2960
06336d82 6046
064a82e9 3809
0657fcde 3344
0666fb4e 4526
069d668b 4463
06cba5d7 2891
06e59db8 6162
06f33221 4675
073b3535 4227
075f92a5 4923
077ee0

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [19]:
import pandas as pd

data = {
    '姓名': ['小明', '小華', '小紅', '小強'],
    '分數': [85, 90, 75, 80]
}

# 創建 DataFrame，並將 '姓名' 列設置為索引
df = pd.DataFrame(data)
df


,姓名,分數
0,小明,85
1,小華,90
2,小紅,75
3,小強,80


In [33]:
# ind = 0
# ind_essay, num_event = event_per_essay[ind]

# cur_df = train_log.loc[id_class.groups[ind_essay]].reset_index().drop("index", axis=1)
# cur_score = train_score.at[ind_essay,"score"]

# ind_event_thousand = 2
# sub_cur_df = cur_df[ind_event_thousand*1000:(ind_event_thousand+1)*1000]
# # aaa = oneHot_eachEvent(sub_cur_df, 1000)

# df = sub_cur_df
# num_desire = 1000

# r_df = df[["down_event"]]
# if(len(df)<num_desire):
#     print("enter padding")
#     padding = { "down_event":["0"]*(num_desire-len(df)) }
#     padding = pd.DataFrame(padding)
#     r_df = pd.concat([r_df, padding], axis=0).reset_index().drop("index", axis=1)

# r_df

# # 製作 r_log (return value)
# r_log = []
# for i in range(num_desire):
#     print(f"enter event {i}")
#     event = r_df.at[i, "down_event"]
#     r_log.append(D_ind[event])


enter padding
enter event 0
enter event 1
enter event 2
enter event 3
enter event 4
enter event 5
enter event 6
enter event 7
enter event 8
enter event 9
enter event 10
enter event 11
enter event 12
enter event 13
enter event 14
enter event 15
enter event 16
enter event 17
enter event 18
enter event 19
enter event 20
enter event 21
enter event 22
enter event 23
enter event 24
enter event 25
enter event 26
enter event 27
enter event 28
enter event 29
enter event 30
enter event 31
enter event 32
enter event 33
enter event 34
enter event 35
enter event 36
enter event 37
enter event 38
enter event 39
enter event 40
enter event 41
enter event 42
enter event 43
enter event 44
enter event 45
enter event 46
enter event 47
enter event 48
enter event 49
enter event 50
enter event 51
enter event 52
enter event 53
enter event 54
enter event 55
enter event 56
enter event 57
enter event 58
enter event 59
enter event 60
enter event 61
enter event 62
enter event 63
enter event 64
enter event 65
enter 

In [22]:
r_df.reset_index().drop("index", axis=1)

,down_event
0,q
1,q
2,q
3,q
4,q
...,...
995,0
996,0
997,0
998,0


In [14]:
np.array(trainLog_2D_augment).shape

(9638, 1000, 131)

In [10]:
trainLog_2D_augment = []
trainScore_2D_augment = []
padding = [0.0]*132
for ind, [ind_essay, num_event] in enumerate(event_per_essay):
  # print(ind_essay,num_event)
  cur_df = train_log.loc[id_class.groups[ind_essay]].reset_index().drop("index", axis=1)
  cur_score = train_score.at[ind_essay,"score"]

  for ind_event_thousand in range(int(num_event)//1000 + 1):
    # print(f"ind_event_thousand = {ind_event_thousand}")
    sub_cur_df = cur_df[ind_event_thousand*1000:(ind_event_thousand+1)*1000]
    trainLog_2D_augment.append( oneHot_eachEvent(sub_cur_df, 1000) )
    trainScore_2D_augment.append([cur_score])

001519c8 2557
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
0022f953 2454
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
0042269b 4136
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
ind_event_thousand = 3
ind_event_thousand = 4
enter padding
0059420b 1556
ind_event_thousand = 0
ind_event_thousand = 1
enter padding
0075873a 2531
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
0081af50 2211
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
0093f095 1765
ind_event_thousand = 0
ind_event_thousand = 1
enter padding
009e23ab 2353
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
enter padding
00e048f1 1585
ind_event_thousand = 0
ind_event_thousand = 1
enter padding
00e1f05a 7826
ind_event_thousand = 0
ind_event_thousand = 1
ind_event_thousand = 2
ind_event_thousand = 3
ind_event_thousand = 4
ind_event_thousand = 

In [18]:
arr_trainScore_2D_augment = np.array(trainScore_2D_augment)
arr_trainScore_2D_augment = arr_trainScore_2D_augment.reshape(-1)
arr_trainScore_2D_augment

array([3.5, 3.5, 3.5, ..., 4. , 4. , 4. ])

In [21]:
arr_trainLog_2D_augment = np.array(trainLog_2D_augment)
arr_trainLog_2D_augment = arr_trainLog_2D_augment.reshape(9638, 1000, 131,1)
arr_trainLog_2D_augment.shape

(9638, 1000, 131, 1)

In [23]:
np.save("/user_data/my/作業/data/trainScore_2D_augment.npy", arr_trainScore_2D_augment)

In [24]:
np.save("/user_data/my/作業/data/trainLog_2D_augment.npy", arr_trainLog_2D_augment)

# Building Model

In [11]:
trainScore_2D_augment = np.load("/user_data/my/作業/data/trainScore_2D_augment.npy")
trainLog_2D_augment   = np.load("/user_data/my/作業/data/trainLog_2D_augment.npy")

In [75]:
print(trainLog_2D_augment.shape)
print(trainScore_2D_augment.shape)

(9638, 1000, 131, 1)
(9638,)


In [108]:
model1D = Sequential(
    [
        Conv1D(10, 3, padding="valid", activation='linear', name = "conv_find"),
        MaxPooling1D(3, padding="valid", strides=1, name = "pool_find"),
        Conv1D( 5, 3, padding="valid", activation='linear', name = "conv_pattern"),
        MaxPooling1D(10, padding="valid", name="pool_pattern"),
        Conv1D( 5, 3, padding="valid", activation='linear', name = "conv_contigious"),
        MaxPooling1D(10, padding="valid", name="pool_contigious"),
        Flatten(name="flatten"),
        Dense(10, activation='linear', name="dense_nn"),
        BatchNormalization(name = "BatchNormalization"),
        Dense(1, activation='linear', name="dense_scoring")
    ],
    "shortCut_vs_score_1D"
)

In [109]:
input_shape = (None, 1000, 131)
model1D.build(input_shape)
model1D.summary()

Model: "shortCut_vs_score_1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_find (Conv1D)          (None, 998, 10)           3940      
                                                                 
 pool_find (MaxPooling1D)    (None, 996, 10)           0         
                                                                 
 conv_pattern (Conv1D)       (None, 994, 5)            155       
                                                                 
 pool_pattern (MaxPooling1D  (None, 99, 5)             0         
 )                                                               
                                                                 
 conv_contigious (Conv1D)    (None, 97, 5)             80        
                                                                 
 pool_contigious (MaxPoolin  (None, 9, 5)              0         
 g1D)                                         

In [110]:
model1D.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['mean_squared_error']
              )

In [111]:
train_feature = trainLog_2D_augment.reshape((9638, 1000, 131))
train_feature.shape

(9638, 1000, 131)